# Prospect VI tool demo with a notebook  

Works at nersc, with master desi environment, which includes prospect (and bokeh 2.1.1)

Simple prospect demo, based on a nightly coadd file from MiniSV2 (andes release)

In [1]:
import os, sys
import numpy as np
from astropy.table import Table
from pathlib import Path
import desispec

# Uncomment this line to use a specific version of prospect
sys.path.insert(0,"/global/homes/a/armengau/prospect/py")
from prospect import utilities,plotframes

In [2]:
# Path to the directory where files are saved
data_path = Path(os.environ['DESI_SPECTRO_REDUX']+"/andes/tiles/70003/20200228")

In [3]:
# Read a coadd file + associated summary of redrock outputs
allcoadds = desispec.io.read_spectra(data_path/"coadd-3-70003-20200228.fits")
allzbest = Table.read(data_path/"zbest-3-70003-20200228.fits",'ZBEST')
print(allcoadds.num_spectra(),"spectra in file")

500 spectra in file


In [4]:
# Select a subsample of spectra to be visually inspected:
zbest_select = allzbest[(allzbest["Z"]>0.6)&(allzbest["Z"]<1)&(allzbest["DELTACHI2"]<50)]
targets = zbest_select["TARGETID"]
coadds_select = allcoadds.select(targets=targets)
print(coadds_select.num_spectra(),"spectra selected")

31 spectra selected


#### Spectral viewer
DO NOT load too many spectra into the viewer as it might overload your web browser (VI sets are currently made of 50 spectra).

In [5]:
help(plotframes.plotspectra)

Help on function plotspectra in module prospect.plotframes:

plotspectra(spectra, nspec=None, startspec=None, zcatalog=None, redrock_cat=None, num_approx_fits=None, with_full_2ndfit=True, model_from_zcat=True, model=None, notebook=False, is_coadded=True, title=None, html_dir=None, with_imaging=True, with_noise=True, with_coaddcam=True, mask_type='DESI_TARGET', with_thumb_tab=True, with_vi_widgets=True, with_thumb_only_page=False, template_dir=None, archetype_fit=False, archetypes_dir=None)
    Main prospect routine. From a set of spectra, creates a bokeh document
    used for VI, to be displayed as an HTML page or within a Jupyter notebook.
    
    Parameters
    ----------
    spectra: input spectra. Supported formats: 1) a 3-band DESI spectra object, with bands 'b', 'r', 'z'. 2) a single-band
        DESI spectra object, bandname 'brz'. 2) a list of 3 frames, associated to the b, r and z bands.
    zcatalog (default None): astropy Table, containing the 'ZBEST' output redrock. Curren

In [ ]:
plotframes.plotspectra(coadds_select, zcatalog=zbest_select, notebook=True, 
                       title='Prospect_demo', 
                       mask_type='CMX_TARGET')